In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib as mpl
from sklearn.linear_model import LogisticRegression
plt.style.use('seaborn')
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
data = pd.read_csv(r"C:\Users\ehj14\Desktop\baseball.csv")
data.head()

In [ ]:
# 현 데이터를 통해 알고자하는 바는 '어떤 지표가 우리 팀을 플레이오프로 이끌었지'이다.
# 따라서('RankPlayoffs', 'RankSeason')은 제거해고자 한다.
data= data.drop('RankPlayoffs', axis =1)
data = data.drop('RankSeason', axis=1)

In [ ]:
# 또한 같은 의미에서 ('Team', 'Year', 'W', 'G') 분석대상과 거리가 멀다.
data = data.drop('Team', axis=1)
data = data.drop('Year', axis=1)
data = data.drop('W', axis=1)
data = data.drop('G', axis=1)

In [ ]:
# 회귀분석에서 문자형 변수는 사용불가하므로, 대체해줘야 한다.
data.League.replace({'AL':0, 'NL':1}, inplace=True)
data.isnull().sum()

In [ ]:
# ('OOBP', 'OSLG')의 경우 결측치가 많은데, 상대방에 대한 기록이 최근에서야 이루어졌기 때문이라고 생각된다.
# 상대방이 못한 경우 역시 플레이오프로 이끌 수 있기 때문에 이를 결측치를 평균값으로 대체해야 한다.
data['OOBP'].fillna(data['OOBP'].mean(), inplace=True)
data['OSLG'].fillna(data['OSLG'].mean(), inplace=True)

In [ ]:
# 설명변수와 종속변수 나누기
x_data = data[['League', 'RS', 'RA', 'OBP', 'SLG', 'BA', 'OOBP', 'OSLG']]
y_data = pd.DataFrame(data['Playoffs'], columns=['Playoffs'])
xy_data = pd.concat([x_data, y_data], axis=1)

In [ ]:
correlation_features = list(x_data)
data[correlation_features].corr()
sns.heatmap(data[correlation_features].corr(),annot=True,linewidths=0.2) 
fig=plt.gcf()
fig.set_size_inches(10,8)
plt.show()

: 상관계수를 살펴보면, RS와 OBP, SLG가 높은 상관관계를 가지고 있다. 일반적으로, 장타를 쳐야 출루를 할 수 있고, 출루를 해야 득점으로 이어질 수 있다는 것을 알고 있다면, 이와 같은 결과는 당연하다.

In [ ]:
# 상관관계가 높은 자료들을 위주로 그래프를 그려보면,
 
graph_1 = sns.jointplot(x='RS', y='SLG', data=data)

<그래프 1> : '득점'과 '장타율'의 관계로, 타자가 공을 쳐야만 득점을 낼 수 있다는 야구 규칙을 살펴본다면, 높은 상관관계를 가지는 것이 당연하다.

In [ ]:
graph_2 = sns.jointplot(x='RA', y='OOBP', data=data)

<그래프2> : '실점'과 '상대방의 출루' 역시 실점은 상대방의 출루가 있어야만 가능하기 때문에 이와같은 그래프가 나온다. 하지만 중간의 직선과 같은 형태의 점들의 잡합이 있는데 이것은 누락된 기록을 평균값으로 대체해주었기 때문에 생긴 것으로 보인다.

In [ ]:
graph_3 = sns.regplot(x=data['OBP'], y = data['BA'])

<그래프3> : '타율'과 '장타율' 역시 장타율의 의미가 쉽게 공이 멀리 날아간 타율을 말하고 있는 것이기 때문에, 종속적인 관계가 확실하여, 높은 상관이 나타나는 것이다.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
warnings.filterwarnings(action='once')

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)
import sklearn as sk
sk.metrics.mean_squared_error(y_train, model.predict(X_train))
model.score(X_test, y_test)
sk.metrics.mean_squared_error(y_test, model.predict(X_test))

: train 데이터와 test 데이터를 비교해볼 때, R-squared와 MSE는 대략적으로 각각 0.89, 0.85, 0.10, 0.14 의 수치를 보이고 있다. 이를 판단해보면, 로지스틱 회귀는 85%의 설명력과 0.14의 오차를 지니고 있다. 
즉, 꽤 설명력이 높다고 할 수 있다.

In [ ]:
print(model.coef_)
print(model.intercept_)

: 각 회귀계수들을 비교해보면,
 
0.18699265 : League
2.10679917 : RS
-2.42946768 :RA
0.69157859 : OBP
0.23721194 : SLG
-0.23691645 : BA
-0.45190743 : OOBP
0.10386695 : OSLG
 
와 같이 나타난다. 로지스틱 회귀에서의 회귀계수의 의미는 해당 변수가 증가할 때, 로그 오즈의 변화량이 각각 회수계수만큼 증가한다는 것이다. 
즉, 선형 회귀와 달리, 종속 변수의 증가량을 나타내지는 않는다. 하지만 로지스틱 회귀에서 역시 계수가 높다는 것은 변화량이 크다는 것을 의미하며, 결과적으로 RS, OBP가 플레이오프 진출을 설명한다고 말할 수 있다.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
 
model = LogisticRegression()
scores = cross_val_score(model, X,y, cv=10)
print(scores)
print(scores.mean())

: 교차검증을 통해 0.89516129 0.87096774 0.84677419 0.91935484 0.90243902 0.86178862 0.84552846 0.86178862 0.91803279 0.82786885 의 결과를 얻을 수 있었다. 
R-squared의 값이 0.82부터 0.91까지 분포되어 있다. 이에 대해서 평균값을 최종 R-sqaured 값으로 한다면, 해당 회귀는 87%의 설명력을 지니고 있고, 
앞서 교차검증을 하지 않은 데이터(train-test)의 값과 비교해 볼 때, 85%에서 87%로 증가한 것을 볼 수 있다.